# 前処理大全 SQL版 (Chapter4)

## はじめに
- データベースはPostgreSQL13です
- 初めに以下のセルを実行してください
- セルに %%sql と記載することでSQLを発行することができます
- jupyterからはdescribeコマンドによるテーブル構造の確認ができないため、テーブル構造を確認する場合はlimitを指定したSELECTなどで代用してください
- 使い慣れたSQLクライアントを使っても問題ありません（接続情報は以下の通り）
  - IPアドレス：Docker Desktopの場合はlocalhost、Docker toolboxの場合は192.168.99.100
  - Port:5432
  - database名: dsdojo_db
  - ユーザ名：padawan
  - パスワード:padawan12345
- 大量出力を行うとJupyterが固まることがあるため、出力件数は制限することを推奨します（設問にも出力件数を記載）
    - 結果確認のために表示させる量を適切にコントロールし、作業を軽快にすすめる技術もデータ加工には求められます
- 大量結果が出力された場合は、ファイルが重くなり以降開けなくなることもあります
    - その場合、作業結果は消えますがファイルをGitHubから取り直してください
    - vimエディタなどで大量出力範囲を削除することもできます
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
%load_ext sql
import os

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# MagicコマンドでSQLを書くための設定
%sql $dsl

'Connected: padawan@dsdojo_db'

# 演習問題

### 004_join/001: マスターテーブルの結合

- 宿泊人数が１名のビジネスホテルの予約レコードを抽出せよ。
- ただし、宿泊人数は、`reserve_tb`の`people_num`に、ビジネスホテルかどうかは、`hotel_tb`の`is_business`に格納されている。

In [15]:
%%sql
SELECT
    *
FROM reserve_tb
INNER JOIN hotel_tb ON reserve_tb.hotel_id = hotel_tb.hotel_id
WHERE reserve_tb.people_num = '1' AND hotel_tb.is_business
LIMIT 5

 * postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,hotel_id_1,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,h_256,34500,C,C-1,38.23729380047828,140.69613145016714,True
r11,h_183,c_2,2016-11-19 12:49:10,2016-12-08,11:00:00,2016-12-11,1,29700,h_183,9900,G,G-4,33.59524814095058,130.6335670204622,True
r13,h_223,c_2,2017-10-19 03:03:30,2017-10-21,09:30:00,2017-10-23,1,137000,h_223,68500,C,C-2,38.32909728396168,140.69816533158763,True
r18,h_132,c_3,2016-10-22 02:18:48,2016-11-12,12:00:00,2016-11-13,1,20400,h_132,20400,C,C-1,38.23184150335627,140.79726757440457,True
r25,h_277,c_4,2016-03-28 07:17:34,2016-04-07,10:30:00,2016-04-10,1,39300,h_277,13100,C,C-1,38.23398539557083,140.79560274996973,True


### 004_join/002: 条件に応じたテーブルの切り替え

- ホテルテーブルのホテルに対してレコメンド候補のホテルを紐づけたテーブルを作成する。
  - レコメンド候補は、以下のルールで選択
    - 同じ`small_area_name`のホテルが20件以上ある場合は、同一の`small_area_name`のテーブルからレコメンド
    - 同じ`small_area_name`のホテルが20件未満の場合は、同一の`big_area_name`のテーブルからレコメンド
- 結果は、`hotel_id`と`recommend_hotel_id`のペアのリストテーブルとする。

In [114]:
%%sql
WITH recommend_area_each_of_hotel_id AS (
    WITH recommend_area_each_of_small_area AS (
        SELECT
            small_area_name,
            CASE
                WHEN COUNT(*)-1 >= 20
                THEN
                    small_area_name
                ELSE
                    big_area_name
                END
            AS recommend_area_name
        FROM hotel_tb
        GROUP BY big_area_name, small_area_name
    )
    SELECT hotel_id, recommend_area_name 
    FROM hotel_tb
    INNER JOIN recommend_area_each_of_small_area ON hotel_tb.small_area_name = recommend_area_each_of_small_area.small_area_name
)
SELECT
    tab_l.hotel_id,
    tab_r.hotel_id AS recommend_hotel_id
FROM recommend_area_each_of_hotel_id tab_l
INNER JOIN recommend_area_each_of_hotel_id tab_r
    ON tab_l.recommend_area_name = tab_r.recommend_area_name
    AND tab_l.hotel_id <> tab_r.hotel_id

 * postgresql://padawan:***@db:5432/dsdojo_db
9364 rows affected.


hotel_id,recommend_hotel_id
h_1,h_292
h_1,h_275
h_1,h_274
h_1,h_265
h_1,h_263
h_1,h_261
h_1,h_221
h_1,h_218
h_1,h_196
h_1,h_129


In [115]:
%%sql
WITH small_area_mst AS (
    SELECT
        small_area_name,
        CASE
            WHEN COUNT(*)-1 >= 20
                THEN small_area_name
                ELSE big_area_name
        END
        AS join_area_id
    FROM hotel_tb
    GROUP BY big_area_name, small_area_name
), recommend_hotel_mst AS (
    SELECT
        big_area_name AS join_area_id,
        hotel_id AS rec_hotel_id
    FROM hotel_tb
    UNION
    SELECT
        small_area_name AS join_area_id,
        hotel_id AS rec_hotel_id
    FROM hotel_tb
)
SELECT
    hotels.hotel_id,
    r_hotel_mst.rec_hotel_id
FROM hotel_tb hotels
INNER JOIN small_area_mst s_area_mst
    ON hotels.small_area_name = s_area_mst.small_area_name
INNER JOIN recommend_hotel_mst r_hotel_mst
    ON s_area_mst.join_area_id = r_hotel_mst.join_area_id
    AND hotels.hotel_id <> r_hotel_mst.rec_hotel_id

 * postgresql://padawan:***@db:5432/dsdojo_db
9364 rows affected.


hotel_id,rec_hotel_id
h_1,h_275
h_1,h_77
h_1,h_79
h_1,h_91
h_1,h_40
h_1,h_221
h_1,h_124
h_1,h_127
h_1,h_103
h_1,h_129


### 004_join/003: 過去データの結合

- 顧客の過去２回前の予約価格をreserve_tbに追加せよ。

In [126]:
%%sql
SELECT
    *,
    LAG(total_price, 2) OVER (PARTITION BY customer_id ORDER BY reserve_datetime) AS before_price
FROM
    reserve_tb
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,before_price
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,None
r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,None
r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,97200
r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,20600
r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,33600
r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,194400
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,68100
r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,36000
r59,h_245,c_10,2016-06-20 11:10:52,2016-07-08,11:00:00,2016-07-10,4,110400,None
r60,h_102,c_10,2016-10-23 01:56:42,2016-11-01,11:00:00,2016-11-03,3,36000,None


- 顧客の過去２回前から現在までの３回分の平均価格をreserve_tbに追加せよ。

In [132]:
%%sql
SELECT
    *,
    CASE WHEN COUNT(total_price) OVER (PARTITION BY customer_id ORDER BY reserve_datetime ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) = 3
    THEN
        SUM(total_price) OVER (PARTITION BY customer_id ORDER BY reserve_datetime ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)
    ELSE
        NULL
    END AS price_sum
FROM reserve_tb
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_sum
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,None
r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,None
r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,151400
r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,248600
r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,296100
r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,298500
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,207600
r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,145500
r59,h_245,c_10,2016-06-20 11:10:52,2016-07-08,11:00:00,2016-07-10,4,110400,None
r60,h_102,c_10,2016-10-23 01:56:42,2016-11-01,11:00:00,2016-11-03,3,36000,None


- 過去３件までの平均値を求めよ。

In [136]:
%%sql
SELECT
    *,
    AVG(total_price)
        OVER (PARTITION BY customer_id ORDER BY reserve_datetime ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) AS price_avg
FROM reserve_tb
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_avg
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,None
r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,97200.000000000000
r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,58900.000000000000
r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,50466.666666666667
r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,82866.666666666667
r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,98700.000000000000
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,99500.000000000000
r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,69200.000000000000
r59,h_245,c_10,2016-06-20 11:10:52,2016-07-08,11:00:00,2016-07-10,4,110400,None
r60,h_102,c_10,2016-10-23 01:56:42,2016-11-01,11:00:00,2016-11-03,3,36000,110400.000000000000


- 過去90日間の平均を求めよ

In [148]:
%%sql
SELECT
    base.*,
    COALESCE( SUM(combine.total_price), 0) AS price_sum
FROM reserve_tb base
LEFT JOIN reserve_tb combine
    ON base.customer_id = combine.customer_id
        AND base.reserve_datetime > combine.reserve_datetime
        AND base.reserve_datetime + cast( '-90 days' as INTERVAL ) <= combine.reserve_datetime
GROUP BY
    base.reserve_id, base.hotel_id, base.customer_id,
    base.reserve_datetime, base.checkin_date, base.checkin_time, base.checkout_date,
    base.people_num, base.total_price
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_sum
r2824,h_5,c_710,2016-11-15 20:28:48,2016-12-10,09:00:00,2016-12-11,3,40500,0
r1370,h_234,c_336,2017-10-22 00:21:07,2017-11-20,11:30:00,2017-11-21,3,16800,0
r1488,h_191,c_366,2016-03-27 19:14:10,2016-04-19,11:00:00,2016-04-20,1,41300,0
r3085,h_80,c_775,2018-01-18 18:27:10,2018-02-05,12:30:00,2018-02-06,3,110100,0
r865,h_61,c_210,2016-07-29 10:12:03,2016-08-05,09:30:00,2016-08-07,3,448800,15400
r2488,h_285,c_622,2016-04-01 15:26:37,2016-04-11,12:30:00,2016-04-14,2,71400,86400
r3205,h_65,c_806,2016-01-02 01:07:26,2016-01-16,10:30:00,2016-01-17,3,25200,0
r1096,h_54,c_267,2016-06-04 11:49:37,2016-06-20,10:00:00,2016-06-21,2,25600,29100
r3509,h_270,c_871,2017-02-03 15:27:41,2017-03-02,12:00:00,2017-03-04,4,66400,231600
r2432,h_255,c_608,2017-02-13 15:34:55,2017-03-15,11:00:00,2017-03-18,3,346500,14800


- GROUP BYは全部列挙する以外に方法はない？
- PostgresにはDATEADDがなかったため、別の書き方で実施した。

### 004_join/004: 全結合

- CROSS JOINはキーを指定せず全てのパターンの結合となる。
- 対象のレコードがない場合にも集計したい場合などに使用する。
- データ数が膨大になるため最低限の範囲で実施することが必要。

In [9]:
%%sql
SELECT
    cus.customer_id,
    mst.year_num,
    mst.month_num,
    SUM(COALESCE(rsv.total_price, 0)) AS total_price_month
FROM customer_tb cus
CROSS JOIN month_mst mst
LEFT JOIN reserve_tb rsv
    ON cus.customer_id = rsv.customer_id
        AND mst.month_first_day <= rsv.checkin_date
        AND mst.month_last_day >= rsv.checkin_date
WHERE mst.month_first_day >= '2017-01-01'
    AND mst.month_first_day >= '2017-04-01'
GROUP BY cus.customer_id, mst.year_num, mst.month_num
ORDER BY cus.customer_id
LIMIT 20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


customer_id,year_num,month_num,total_price_month
c_1,2017,4,0
c_1,2017,5,0
c_1,2017,6,0
c_1,2017,7,0
c_1,2017,8,0
c_1,2017,9,68100
c_1,2017,10,0
c_1,2017,11,0
c_1,2017,12,36000
c_1,2018,1,103500
